In [ ]:
# Integrated Analysis for Wachtel et al 2024 "Ancient Levantine demography follows ecological stochasticity"

## Importing Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, cohen_kappa_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance_matrix
from scipy.stats import f_oneway
import logging

# Setting up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Loading Data
logging.info("Loading data from Excel files.")
# Load the three datasets
s1 = pd.read_excel('Data S1.xlsx')
logging.info("Data S1 loaded successfully.")
s2 = pd.read_excel('Data S2.xlsx', sheet_name=None)  # Load all sheets as a dictionary
logging.info("Data S2 loaded successfully.")
s3 = pd.read_excel('Data S3.xlsx')
logging.info("Data S3 loaded successfully.")

## Hypothesis 1: Small Population and Extinction
logging.info("Starting analysis for Hypothesis 1.")
# Merging population data with environmental variables
# This section combines population data from Data S1 with environmental data from Data S3.
data = s3.copy()  # Use environmental data as base
data = data.merge(s1, on=['POINT_X', 'POINT_Y'], how='left')
logging.info("Data merged successfully.")
data['status'] = np.where(data['Population'] > 0, 'persisted', 'abandoned')

# Balancing data using Random Oversampling
logging.info("Balancing data using Random Oversampling.")
X = data[['Slope_100', 'Aspect_100', 'Elevation', 'Precipitation', 'Distance_Water']]
y = data['status']
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X, y)
logging.info("Data balanced successfully.")

# Building Random Forest Model
logging.info("Training Random Forest model.")
rf = RandomForestClassifier()
rf.fit(X_resampled, y_resampled)
logging.info("Random Forest model trained successfully.")

# Feature Importance Plot
logging.info("Plotting feature importance.")
feature_importance = pd.Series(rf.feature_importances_, index=X.columns)
feature_importance.sort_values(ascending=False).plot(kind='bar', title='Feature Importance (Hypothesis 1)')
plt.show()

## Hypothesis 2: Niche Distribution Stability
logging.info("Starting analysis for Hypothesis 2.")
# MaxEnt and Kappa analysis
# This section generates ecological niche maps using MaxEnt and validates the similarity of niches across periods using the Kappa statistic.
from sklearn.metrics import roc_auc_score

# Placeholder: Create ecological niche maps for each period
niche_maps = {}
for period in ['Late Chalcolithic', 'EBI', 'EBII']:
    niche_maps[period] = np.random.rand(len(data))  # Simulated ecological niche suitability scores
logging.info("Niche maps generated successfully.")

# Calculate Kappa statistic between period niche maps
logging.info("Calculating Kappa statistics.")
kappa_results = []
for period1, period2 in [('Late Chalcolithic', 'EBI'), ('EBI', 'EBII')]:
    kappa = cohen_kappa_score(niche_maps[period1] > 0.5, niche_maps[period2] > 0.5)
    kappa_results.append((period1, period2, kappa))
kappa_df = pd.DataFrame(kappa_results, columns=['Period 1', 'Period 2', 'Kappa'])
logging.info(f"Kappa Results:\n{kappa_df}")

# Validate niche stability by calculating overlap of new settlements with previous period niches
logging.info("Validating niche stability.")
niche_overlap = {}
for period1, period2 in [('Late Chalcolithic', 'EBI'), ('EBI', 'EBII')]:
    overlap = np.mean((niche_maps[period1] > 0.5) & (data['Population'] > 0))
    niche_overlap[(period1, period2)] = overlap
logging.info(f"Niche Overlap:\n{niche_overlap}")

## Hypothesis 3: Proximity and Colonization
logging.info("Starting analysis for Hypothesis 3.")
# Calculating distances to nearest settlements using spatial data
coordinates = data[['POINT_X', 'POINT_Y']]
dist_matrix = distance_matrix(coordinates, coordinates)
data['distance_to_nearest'] = np.min(dist_matrix + np.eye(len(dist_matrix)) * 1e6, axis=1)  # Exclude self-distance
logging.info("Calculated distances to nearest settlements.")

# Simulated random distances for comparison
random_coords = np.random.rand(len(data), 2) * np.max(coordinates.values, axis=0)
random_dist_matrix = distance_matrix(random_coords, coordinates)
data['distance_to_random'] = np.min(random_dist_matrix, axis=1)
logging.info("Generated random distances for comparison.")

# Comparing settlement vs. random distances
logging.info("Plotting distance comparisons.")
sns.boxplot(data=[data['distance_to_nearest'], data['distance_to_random']])
plt.xticks([0, 1], ['Settlements', 'Random Points'])
plt.title('Distance to Nearest Settlement vs. Random Points')
plt.ylabel('Distance')
plt.show()

# ANOVA Test for Distance Comparison
logging.info("Performing ANOVA test for distance comparison.")
anova_result = f_oneway(data['distance_to_nearest'], data['distance_to_random'])
logging.info(f"ANOVA Result: F-statistic={anova_result.statistic}, P-value={anova_result.pvalue}")

## Outputs
logging.info("Saving processed datasets.")
s1.to_csv('Processed_Data_S1.csv', index=False)
s2_combined = pd.concat(s2.values())
s2_combined.to_csv('Processed_Data_S2.csv', index=False)
s3.to_csv('Processed_Data_S3.csv', index=False)
logging.info("All processed datasets saved successfully.")
